In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

In [7]:
def get_creds(SCOPES):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        print("path exists")
        print(creds)
        # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "../credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    print(creds)
    return creds

In [8]:
def get_events(creds):
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")
  return events  

In [9]:
def add_event(event, creds):
  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))
    
  except HttpError as error:
    print(f"An error occurred: {error}")

In [10]:
creds = get_creds(SCOPES)

path exists


In [12]:
events = get_events(creds)

Getting the upcoming 10 events
2024-05-24T09:00:00-04:00 Rdv Carla - Brushing
2024-05-24T10:30:00-04:00 Rdv Laura - Permanente
2024-05-24T12:00:00-04:00 Rdv - Ismail ajouté automatiquement
2024-05-24T13:00:00-04:00 Rdv Melika - Soins
2024-05-24T14:00:00-04:00 Rdv - Badr ajouté automatiquement


In [13]:
event = {
  'summary': 'Rdv - Badr ajouté automatiquement',
#   'location': '800 Howard St., San Francisco, CA 94103',
  'description': 'Coupe homme tondeuse',
  'start': {
    'dateTime': '2024-05-24T11:00:00-07:00',
    'timeZone': 'Europe/Paris',
  },
  'end': {
    'dateTime': '2024-05-24T11:30:00-07:00',
    'timeZone': 'Europe/Paris',
  },
#   'recurrence': [
#     'RRULE:FREQ=DAILY;COUNT=2'
#   ],
  'attendees': [
    {'email': 'badrelidrissimokdad12@gmail.com'},
    # {'email': 'sbrin@example.com'},
  ],
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

In [14]:
add_event(event, creds)

Event created: https://www.google.com/calendar/event?eid=NGY0N2MxMGttOWx2OHY5c2dhOTNzbWJqcjggY29udGFjdHNrYWJhZGlzQG0
